In [13]:
!pip3 install plotly
!pip3 install nbformat


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for nbformat from https://files.pythonhosted.org/packages/a9/82/0340caa499416c78e5d8f5f05947ae4bc3cba53c9f038ab6e9ed964e22f1/nbformat-5.10.4-py3-none-any.whl.metadata
  Obtaining dependency information for fastjsonschema>=2.15 from https://files.pythonhosted.org/packages/9c/b9/79691036d4a8f9857e74d1728b23f34f583b81350a27492edda58d5604e1/fastjsonschema-2.19.1-py3-none-any.whl.metadata
  Obtaining dependency information for jsonschema>=2.6 from https://files.pythonhosted.org/packages/39/9d/b035d024c62c85f2e2d4806a59ca7b8520307f34e0932fbc8cc75fe7b2d9/jsonschema-4.21.1-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=22.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa6703fcf4a79018f/attrs-23.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for jsonschema-sp

In [1]:
import duckdb 
import pandas 
import plotly.express as px

In [54]:
con = duckdb.connect()
attach_results = con.sql("ATTACH 'benchmark_log_python_testing.db' as sqlite_db (TYPE SQLITE)")
ctas_results = con.sql("create or replace table duckdb_results as select * from sqlite_db.results")
results = con.sql("""
    with dummy_results as (
        from duckdb_results 
        union all
        from duckdb_results
        select * replace (
            '{"duckdb_version": "v0.0.2"}' as scenario,
            time * 2 as time
        )
        union all
        from duckdb_results
        select * replace (
            '{"duckdb_version": "v0.0.1"}' as scenario,
            time * 3 + time * random() as time
        )
    ), max_version as (
        from dummy_results 
        select max((scenario::json ->> 'duckdb_version').replace('v','')) as max_version
    )
    from dummy_results
    join max_version on 1=1
    select 
        * exclude time,
        median(time) over (partition by run_id, benchmark, scenario) as "Time (seconds)",
        (scenario::json ->> 'duckdb_version').replace('v','') as "DuckDB Version",
        case when benchmark ilike '%queries%' or benchmark ilike '%enums%' then 'Analysis'
             when benchmark ilike '%csv%' then 'CSV Import' 
             when benchmark ilike '%scan%' or benchmark ilike '%query pandas%' then 'Scan other formats' 
             when benchmark ilike '%export%' then 'Export'
        end as benchmark_type,
        case when benchmark_type = 'CSV Import' then 1
             when benchmark_type = 'Scan other formats' then 2
             when benchmark_type = 'Analysis' then 3
             when benchmark_type = 'Export' then 4
             end as benchmark_type_order,
        benchmark_type as "Benchmark Type",
        sum(time) over (partition by "DuckDB Version") as total_time_by_version,
        sum(case when (scenario::json ->> 'duckdb_version').replace('v','') = max_version.max_version then time else 0 end) over () as total_time_max_version,
        time / total_time_max_version as "Time Relative to Latest Version"
    order by benchmark, run_id, scenario, repeat_id""").df()
results

,run_id,repeat_id,benchmark,scenario,max_version,Time (seconds),DuckDB Version,benchmark_type,benchmark_type_order,Benchmark Type,total_time_by_version,total_time_max_version,Time Relative to Latest Version
0,1,0,001 Query pandas,"{""duckdb_version"": ""v0.0.1""}",0.10.1,0.038065,0.0.1,Scan other formats,2,Scan other formats,177.093098,51.053585,0.000746
1,1,0,001 Query pandas,"{""duckdb_version"": ""v0.0.2""}",0.10.1,0.021431,0.0.2,Scan other formats,2,Scan other formats,102.107171,51.053585,0.000420
2,1,0,001 Query pandas,"{""duckdb_version"": ""v0.10.1""}",0.10.1,0.010716,0.10.1,Scan other formats,2,Scan other formats,51.053585,51.053585,0.000210
3,1,0,002 Create table from csv,"{""duckdb_version"": ""v0.0.1""}",0.10.1,7.494020,0.0.1,CSV Import,1,CSV Import,177.093098,51.053585,0.146787
4,1,0,002 Create table from csv,"{""duckdb_version"": ""v0.0.2""}",0.10.1,4.804336,0.0.2,CSV Import,1,CSV Import,102.107171,51.053585,0.094104
5,1,0,002 Create table from csv,"{""duckdb_version"": ""v0.10.1""}",0.10.1,2.402168,0.10.1,CSV Import,1,CSV Import,51.053585,51.053585,0.047052
6,1,0,003 Convert to Enums,"{""duckdb_version"": ""v0.0.1""}",0.10.1,4.370156,0.0.1,Analysis,3,Analysis,177.093098,51.053585,0.085599
7,1,0,003 Convert to Enums,"{""duckdb_version"": ""v0.0.2""}",0.10.1,2.463481,0.0.2,Analysis,3,Analysis,102.107171,51.053585,0.048253
8,1,0,003 Convert to Enums,"{""duckdb_version"": ""v0.10.1""}",0.10.1,1.231741,0.10.1,Analysis,3,Analysis,51.053585,51.053585,0.024126
9,1,0,004 Group by queries,"{""duckdb_version"": ""v0.0.1""}",0.10.1,7.454709,0.0.1,Analysis,3,Analysis,177.093098,51.053585,0.146017


In [55]:
fig = px.area(
    results,
    x="DuckDB Version",
    y="Time (seconds)",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis","Export"]},
    template='plotly_white'
)
fig.show()

In [53]:
fig = px.area(
    results,
    x="DuckDB Version",
    y="Time Relative to Latest Version",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis","Export"]},
    template='plotly_white'
)
fig.show()

## Brainstorming
- Total time area plot
- Relative time area plot (with current as 1, scaling the rest off of that)
    - Grouping related operations together
        - Maybe grouping by type (export to pandas)
        - Maybe grouping by task (group by tests)
        - Group by "importing, querying, exporting" to show the whole burger idea
    - Likely need to filter things down
    - Choosing between Pandas and Arrow whenever Arrow was faster
- Annotating the charts to show what improved with each version

- Max data scale plot
    - Likely need more than just 2 datasets (10 million, 100 million, 1 billion, 10 billion?) Likely unable to generate 10 billion locally with this script.
    - Probably want a column chart with group by and join as the 2 types of bars (alternating group by / join for each version showing the max it can handle? Or maybe a line chart)


# TODO: Pre-process into median time